# OMOP CDM (The Observational Medical Outcomes Partnership (OMOP) Common Data Model (CDM))

As we learned from previous modules, with the structured database and unified standards for medicial terms, EHR data will be well-organized and used through the whole health system. However, different health systems such as `Mayo Clinic Health System` and `Sanford Health` might apply with different standards and data organization methodologies. It will be tough for data and information exchange between different applications and data sources.

A [common data model](https://en.wikipedia.org/wiki/Common_data_model) in health care context is able to to standardize logical infrastructure of EHRs and allows data to be sent to and received from different applications and data sources. In this module, we will focus on a popular CDM called `OMOP CDM`.

## 1. Introduction to OMOP

The open-community standard OMOP CDM is intended to provide standards for how to structure and record observational data and to improve the efficiency of analyzing such observational data. It's maintained by [Observational Health Data Sciences and Informatics consortium (OHDSI)](https://www.ohdsi.org/) and applied by health systems all around the world. 

The advantages of converting patient-level data to an OMOP CDM specification are clear:

* Only a single definition of a study query is needed, that can be executed across multiple and diverse health systems. This makes research very **efficient, transparent, reproducible and scalable**. 
* **Interoperability** between health institutions, which allows for easy comparisons. 
* **Extensive list of standardized Health Concepts and Hierarchies**, such as conditions and drugs, as provided by the OHDSI standardized vocabularies. 
* **Suite of OHDSI tools** available in all the processes, comprising tools that range from the beginning of the transformation of source data to large scale analytics and research. 
* **More health data availability as a community**. This is helpful in the case of rare conditions, since more volume of information can be considered. In the beginning of 2022, the OHDSI data network comprised 810 million unique patient records, from 331 data sources. 
* The source information is not lost and can be stored in the CDM tables, which allows **local and distributed research to be conducted using the same model**.

A central component of the OMOP CDM is the `OHDSI standardized vocabularies`. The OHDSI standardized vocabularies provide the OMOP CDM with a set of standard terms that can be used throughout different OMOP clinical domains and allow for standardized analytics when developing exposure and outcome phenotypes and additional features.


## 2. OMOP CDM standards

The image shown below tells us how are the EHRs are structured in OMOP CDM standards. 

![OMOP CDM concepts](https://www.ohdsi.org/wp-content/uploads/2021/09/OHDSI-OurJourney2021-34-35-OMOP-CDM-1024x656.png)

### 2.1 Standarized tables

Based on OMOMP CDM, EHRs can be categorized into below 6 domains.

* **Clinical Data Tables**
    The domain contains all clinical information such as patients, diagnoses, procedures, medications and etc.

* **Health System Data Tables**
    The domain contains all information about specific health system such as providers, locations and etc.

* **Health Economics Data Tables**
    The domain contains all economical information about the health system. 

* **Standardized Derived Elements**
    The domain contains extra information for clinical events from a different perspective. 

* **Metadata Tables**
    The domain contains the metadata of the database.

* **Vocabulary Tables**
    The domain contains the vocabularies used in OMOP CDM. 
    
    
According to [The Book of OHDSI](https://ohdsi.github.io/TheBookOfOhdsi/), chapter 4.2.6, variable names across all tables follow one convention:

|Notation|Description|
|:---|:---|
|\[Event\]_ID|Unique identifier for each record, which serves as a foreign keys establishing relationships across Event tables. For example, PERSON_ID uniquely identifies each individual. VISIT_OCCURRENCE_ID uniquely identifies a Visit.|
|\[Event\]_CONCEPT_ID|Foreign key to a Standard Concept record in the CONCEPT reference table. This is the main representation of the Event, serving as the primary basis for all standardized analytics. For example, CONDITION_CONCEPT_ID = 31967 contains the reference value for the SNOMED concept of “Nausea”.|
|\[Event\]_SOURCE_CONCEPT_ID|Foreign key to a record in the CONCEPT reference table. This Concept is the equivalent of the Source Value (below), and it may happen to be a Standard Concept, at which point it would be identical to the \[Event\]_CONCEPT_ID, or another non-standard concept. For example, CONDITION_SOURCE_CONCEPT_ID = 45431665 denotes the concept of “Nausea” in the Read terminology, and the analogous CONDITION_CONCEPT_ID is the Standard SNOMED-CT Concept 31967. The use of Source Concepts for standard analytics applications is discouraged since only Standard Concepts represent the semantic content of an Event in a unambiguous way and therefore Source Concepts are not interoperable.|
|\[Event\]_TYPE_CONCEPT_ID|Foreign key to a record in the CONCEPT reference table, representing the origin of the source information, standardized within the Standardized Vocabularies. Note that despite the field name this is not a type of an Event, or type of a Concept, but declares the capture mechanism that created this record. For example, DRUG_TYPE_CONCEPT_ID discriminates if a Drug record was derived from a dispensing Event in the pharmacy (“Pharmacy dispensing”) or from an e-prescribing application (“Prescription written”)|
|\[Event\]_SOURCE_VALUE|Verbatim code or free text string reflecting how this Event was represented in the source data. Its use is discouraged for standard analytics applications, as these Source Values are not harmonized across data sources. For example, CONDITION_SOURCE_VALUE might contain a record of “78702”, corresponding to ICD-9 code 787.02 written in a notation omitting the dot.|
    

To get more details about each table, please refer to [OMOP CDM wiki v5.4](https://ohdsi.github.io/CommonDataModel/cdm54.html)



## 3. OMOP CDM ETL

To obtain raw data from the OMOP CDM, an extract, transform, and load (ETL) process must be created. The OMOP CDM ETL process is intended to restructure data and map to Standardized Vocabularies. Generally, this process is executed through a group of automated scripts, such as `Python` or `SQL` scripts. The OMOP CDM ETL process can be repeated whenever the source data is updated.

A general ETL workflow is defined as image below:

![OMOP CDM ETL](https://github.com/Prisma-pResearch/ic3-online-documentation/blob/main/websites/resources/OMOP/omop_ETL.png?raw=true)

Here are the tools used in ETL process:

- [WhiteRabbit](https://github.com/OHDSI/WhiteRabbit): scans the source database and creates a report with the information associated with each table, while also extracting some simple analytics about the source fields.
- [Rabbit In a Hat](https://github.com/OHDSI/WhiteRabbit): relies on the generated report by WhiteRabbit and it is where the mapping between source and target tables is done. 
- [Athena](https://athena.ohdsi.org/): used to fetch the Vocabularies. Note that some may require a license, such as the CPT4 vocabulary, for which an UMLS account is needed.
- [Usagi](https://github.com/OHDSI/Usagi): to map source names that do not have a corresponding standard concept, which happens for example when using a specific term in a given language, such as Portuguese. Alongside with a translation mechanism between Portuguese to English, Usagi can be used to find a similar condition in the current concepts and register it in the CDM.
- [DataQualityDashboards](https://github.com/OHDSI/DataQualityDashboard): Once source data is converted to the OMOP CDM tables, quality metrics can be computed to assure that the process of standardization occurred without errors, for example, to check the plausibility of the values inserted. If the data does not meet the quality criteria expected, some changes need to be made in the previous steps, which places ETL as an iterative process.


## 4. Standardized Vocabulary Mapping

### 4.1 Domains and Vocabularies

The Standardized Vocabularies are organized into `Domains` and `Vocabularies`. Domains refer to the nature or type of a clinical entity. Vocabularies are sets of concepts imported from an external national or international existing standards such as RxNorm, ICD-9 codes and etc. 

> Note: There is no one-to-one relationship between domains and vocabularies (as shown below). Some vocabularies are very broad, such as SNOMED or Read, and contain concepts of all medical domains. Other vocabularies are specific for a certain domain, such as RxNorm for Drugs or ICD9Proc for Procedures.

![Domains and Vocabs](./Images/domainsandvocabs.jpg)

### 4.2 Standard, Classification, and Source Concepts

Standardized codes in a domain can originate from a variety of vocabularies, and many times these codes have redundant meanings. To clarify this potentially confusing issue, each standardized code is assigned one of the following "concept" designations:

* `Standard Concept` (standard_concept = 'S')
This designation is for the “official” concept that represents a specific clinical entity in OMOP CDM standardized tables. The ID for the Standard Concept is written in the *_concept_id field. In general, Standard Concepts originate in vocabularies that are well-respected and provide well-defined concepts and comprehensive coverage of the domain.

<details>
<summary> <u> Load Details </u> </summary>

*For example in the *Condition* domain, this is achieved through the SNOMED vocabulary. If no comprehensive list of available entities is available in a certain domain, such as the **Device** domain, Standard Concepts come from a variety of different Vocabularies. Same is true for the **Procedure** and **Visit** domains.*


</details>
    
* `Classification Concepts` (standard_concept = 'C')
This designation is for concepts that have a hierarchical relationship to Standard Concepts and can query those concepts through the use of records from the CONCEPT_ANCESTOR table. Classification Concepts themselves, though, are not present in data tables. 

<details>
<summary> <u> Load Details </u> </summary>

*For example, the concept 4283987 “ANTICOAGULANTS” of the Vocabulary “VA Class” cannot appear in the DRUG_EXPOSURE or DRUG_ERA tables, but its descendant concepts that have the Concept Class “Ingredient”, “Clinical Drug” or “Branded Drug” can.*

*Classification Concepts may be sourced from different Vocabularies than the Standard Concepts. Note that Classification Concepts are not unique. For example, there are Concepts for the Drug Class “Anticoagulants” coming from the NDF-RT, VA Class, ETC and ATC Vocabularies. Also note that the membership depends on the Vocabulary. In most cases the membership list of equivalent Classification Concepts are similar or identical, but the medical science does not provide a generally agreed upon standard definition of these classes.*

</details>

* `Source Concepts` (standard_concept = NULL)
This designation is for concepts that do not fall into the two preceding categories. Because the designation of a concept can change, a concept from the two preceding categories could be deprecated and change into a Source Concept.

<details >
<summary> <u> Load Details </u> </summary>

*Source Concepts can only appear in the *_source_concept_id fields of the data tables. They represent the code in the source data. Each Source Concept is mapped to one or more Standard Concepts during the ETL process. If no mapping is available, the Standard Concept with the concept_id = 0 is written into the *_concept_id field.*
    
</details>

For all Concepts in a domain, this creates the following logical structure:

![Concepts logic](./Images/concept_structure.jpg)


To get more details about OHDSI vocabulary, please visit [here](https://github.com/OHDSI/Vocabulary-v5.0/wiki).

Once we get the mapping ideas for CDM and Vocabularies, we will be able to apply `Python` to implement the mapping operations. The whole process is very complicated, so we will show some POC(proof of concepts) examples here.

## 5. Clinical Data Mapping 

## 5.1 Target Table: Person
<h3 style='color:green'> About :</h3>
</br>
<div style="text-align: justify">
  This table is used to identify all the patients in the database and contains records with unique identifiers for every patient (including demograpic information). Each record in this table corresponds to independent person and his/her details.
</div>

<h3 style='color:green'>ETL Coversions :</h3>
</br>
<div style="text-align: justify">
In the PERSON table, you will find a record for every patient who appears in the database (unless that record fails ETL data quality standards). Even if a Person has no Events, the Person should still have a record. If a Person has Events contributed from multiple data sources, the data must be reconciled so that each Person has only one record. The content in BIRTH_DATETIME should be the same as the content in BIRTH_DAY, BIRTH_MONTH, and BIRTH_YEAR.
</div>
    
Refer to [click here](https://ohdsi.github.io/CommonDataModel/cdm54.html#PERSON) for more details about Person table


<h3 style='color:green'>Source Table : <h3 style='color:red;align:inline'>Encounters</h3></h3>

<h3 style='color:green'>About :</h3>
</br>
<div style="text-align: justify">
Encounters table is the source table which holds fields like patient_deiden_id, sex, race, ethnicity, birth_date which needs to be mapped to CDM Person table as target. There are also other fields in encounters source which will be mapped to other target tables like Visit_Occurence, Observation later.
</div>

<h3 style='color:green'>Mapping :</h3>

![encounter_2_person](./Images/encounter_2_person.png)

In [1]:
###Code 

import pandas as pd
import numpy as np
import os
from datetime import datetime

#Parse dates of this format from input file to log entries in target table
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

encounters = pd.read_csv('./SampleDatasets/encounters_sample.csv', usecols=["patient_deiden_id", "sex","race","ethnicity","birth_date"],parse_dates=['birth_date'], date_parser=dateparse)

## The first 10 rows of source table
encounters.head(10)

,sex,race,birth_date,ethnicity,patient_deiden_id
0,FEMALE,WHITE,1932-03-21,NOT HISPANIC,114927
1,MALE,WHITE,1947-12-04,NOT HISPANIC,114920
2,MALE,WHITE,1934-05-25,NOT HISPANIC,114985
3,FEMALE,WHITE,1944-09-01,NOT HISPANIC,114930
4,MALE,WHITE,1945-09-25,NOT HISPANIC,114931
5,FEMALE,OTHER,2000-03-20,HISPANIC,114917
6,FEMALE,WHITE,1978-08-30,NOT HISPANIC,114977
7,MALE,WHITE,1938-11-20,NOT HISPANIC,114918
8,MALE,WHITE,1938-11-20,NOT HISPANIC,114918
9,FEMALE,BLACK,1979-12-23,NOT HISPANIC,114978


In [2]:
# Dictionary to bind gender_concept_id to respective gender while populating target table values
gender={'FEMALE': 8532, 'MALE': 8507, 'AMBIGUOUS': 8570, 'Feminine gender': 45766035, 'Gender finding': 4268709, 'Gender unknown': 45518388, 'Gender unspecified': 45454912, 'Male': 45438358, 'Masculine gender': 45766034, 'Non-binary gender': 42689678, 'OTHER': 8521, 'Surgically transgendered transsexual': 4234363, 'Surgically transgendered transsexual, female-to-male': 4231242, 'Surgically transgendered transsexual, male-to-female': 4251434, 'Transgender identity': 36712829, 'UNKNOWN': 8551}

#Dictionary to bind ethnicity_concept_id to respective ethnicity while populating target table values
ethnicity={"HISPANIC":38003563, "LATINO":38003563,"NOT HISPANIC":38003564, "NOT LATINO":38003564}

#Dictionary to bind Race_concept_id to respective race while populating target table values
race={"AMERICAN INDIAN":3421,"ASIAN": 3422,"BLACK":3423,"OTHER":3424,"MULTIRACIAL":3425,"UNKNOWN":3426,"WHITE":3427}

#Target table mapping with ETL conversions

person=pd.DataFrame().assign(person_id=encounters['patient_deiden_id'], person_source_value=encounters['patient_deiden_id'],gender_concept_id=encounters['sex'].str.extract(fr"({'|'.join(gender.keys())})", expand=False).map(gender),
                       gender_source_value= encounters['sex'],race_concept_id=encounters['race'].str.extract(fr"({'|'.join(race.keys())})", expand=False).map(race),race_source_value=encounters['race'],ethnicity_concept_id=encounters['ethnicity'].str.extract(fr"({'|'.join(ethnicity.keys())})", expand=False).map(ethnicity),ethnicity_source_value=encounters['ethnicity'],year_of_birth=encounters['birth_date'].dt.year,month_of_birth=encounters['birth_date'].dt.month,day_of_birth=encounters['birth_date'].dt.day,birth_datetime=encounters['birth_date'].dt.strftime("%Y-%m-%d %H:%M:%S"))

## The first 10 rows of target table
person.head(10)

,person_id,person_source_value,gender_concept_id,gender_source_value,race_concept_id,race_source_value,ethnicity_concept_id,ethnicity_source_value,year_of_birth,month_of_birth,day_of_birth,birth_datetime
0,114927,114927,8532,FEMALE,3427,WHITE,38003564.0,NOT HISPANIC,1932,3,21,1932-03-21 00:00:00
1,114920,114920,8507,MALE,3427,WHITE,38003564.0,NOT HISPANIC,1947,12,4,1947-12-04 00:00:00
2,114985,114985,8507,MALE,3427,WHITE,38003564.0,NOT HISPANIC,1934,5,25,1934-05-25 00:00:00
3,114930,114930,8532,FEMALE,3427,WHITE,38003564.0,NOT HISPANIC,1944,9,1,1944-09-01 00:00:00
4,114931,114931,8507,MALE,3427,WHITE,38003564.0,NOT HISPANIC,1945,9,25,1945-09-25 00:00:00
5,114917,114917,8532,FEMALE,3424,OTHER,38003563.0,HISPANIC,2000,3,20,2000-03-20 00:00:00
6,114977,114977,8532,FEMALE,3427,WHITE,38003564.0,NOT HISPANIC,1978,8,30,1978-08-30 00:00:00
7,114918,114918,8507,MALE,3427,WHITE,38003564.0,NOT HISPANIC,1938,11,20,1938-11-20 00:00:00
8,114918,114918,8507,MALE,3427,WHITE,38003564.0,NOT HISPANIC,1938,11,20,1938-11-20 00:00:00
9,114978,114978,8532,FEMALE,3423,BLACK,38003564.0,NOT HISPANIC,1979,12,23,1979-12-23 00:00:00


## 5.2 Target Table: Observation 

<h3 style='color:green'> About :</h3>
</br>
<div style="text-align: justify">

In the OBSERVATION table, you will find data about a person that was discovered through clinical exams, questions, or procedures. If data cannot be categorized in other domains—for example, social and lifestyle facts or family history—then that data will be recorded in the OBSERVATION table. The types of observations in this table would include details from the patient about healthcare habits, a request for a specific kind of treatment, family and medical history, etc. The OBSERVATION table would not include data that requires a standardized test; that data would be included in the MEASUREMENT table. Additionally, the OBSERVATION table would not include symptoms or disease diagnoses; that data would be included in the CONDITION_OCCURRENCE table. 
</div>
<h3 style='color:green'>ETL Coversions :</h3>
</br>
<div style="text-align: justify">

If a Record has Source Values that map to a domain other than Condition, Procedure, Drug, Measurement, or Device, then that record should be included in the OBSERVATION table. Observations can be recorded as an attribute paired with a value, with the attribute defining the Observation Concept and the value defining the clinical fact. The clinical fact can be any of the following: a Concept (in VALUE_AS_CONCEPT), a number (VALUE_AS_NUMBER), a verbatim string (VALUE_AS_STRING), or a date and time (VALUE_AS_DATETIME). Observations do not yield explicit results, but the clinical fact can be expressed as separate from the attribute (i.e., the Observation Concept) that is defined in the VALUE_AS fields. Ideally, Observations should be positive assertions with a Yes value (concept_id=4188539), despite the null value being equivalent.
</div>

Refer to [click here](https://ohdsi.github.io/CommonDataModel/cdm54.html#OBSERVATION) for more details about Person table

<h3 style='color:green'>Source Table : <h3 style='color:red;align:inline'>Morse</h3></h3>

<h3 style='color:green'>About :</h3>
</br>
<div style="text-align: justify">
Morse table is the source table which holds fields like morse_datetime, measurement name and value which needs to be mapped to CDM Observation table as target.
</div>

<h3 style='color:green'>Mapping :</h3>

![morse_2_observation](./Images/morse_2_obs.png)

In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

#Parse dates of this format from input file to log entries in target table
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

#Source table
morse = pd.read_csv('./SampleDatasets/morse_sample.csv',usecols=["patient_deiden_id", "morse_datetime","measurement_name","measurement_value","merged_enc_id","encounter_deiden_id"],parse_dates=['morse_datetime'], date_parser=dateparse)
morse.head(15)

,patient_deiden_id,morse_datetime,measurement_name,measurement_value,merged_enc_id,encounter_deiden_id
0,108386,2014-10-16 14:25:00,morse_ambulatory_aids,0,108386_m_1,12524022
1,108386,2014-10-16 19:41:00,morse_mental_status,0,108386_m_1,12524022
2,108386,2014-10-16 14:25:00,morse_intravenous,20,108386_m_1,12524022
3,108386,2014-10-16 19:41:00,morse_score,20,108386_m_1,12524022
4,108386,2014-10-15 19:02:00,morse_secondary_dx,0,108386_m_1,12524022
5,108386,2014-10-19 12:30:00,morse_gait_transferring,0,108386_m_1,12524022
6,108386,2014-10-18 11:11:00,morse_fall_hx,0,108386_m_1,12524022
7,108386,2014-10-17 09:15:00,fall_risk_indicator,Yes,108386_m_1,12524022
8,108395,2015-02-04 21:32:00,morse_mental_status,0,108395_m_2,12524030
9,108395,2015-02-04 21:32:00,morse_score,0,108395_m_2,12524030


In [4]:
#Dictionaries to map concept_ids to respective field values in target table from source
obs_conc_id = {"fall_risk_indicator":3521695, "morse_ambulatory_aids":2000018739,"morse_fall_hx":2000018740,"morse_gait_transferring":2000018741,"morse_intravenous":2000018742,"morse_mental_status":2000018743,"morse_score":2000018744,"morse_secondary_dx":2000018745}
value_as_concept_id = {"-999":3542361,"0.0":3542362,"0":3542363,"10.0":3542364,"15":3542365,"15.0":3542366,"20":3542367,"35":3542368,
                      "45":3542369,"60":3542370,"70":3542371,"Yes":3542372}

#Target table mapping with ETL conversions
obs=pd.DataFrame().assign(person_id=morse['patient_deiden_id'], visit_occurrence_id=morse['merged_enc_id'], visit_detail_id=morse['encounter_deiden_id'],
observation_date= morse['morse_datetime'].dt.strftime("%Y-%m-%d"), observation_datetime=morse['morse_datetime'], observation_concept_id=morse['measurement_name'].str.extract(fr"({'|'.join(obs_conc_id.keys())})", expand=False).map(obs_conc_id),
    observation_source_value=morse['measurement_name'], value_as_string=morse['measurement_value'],
    value_as_concept_id=morse['measurement_value'].str.extract(fr"({'|'.join(value_as_concept_id.keys())})", expand=False).map(value_as_concept_id),
    value_source_value=morse['measurement_value'], observation_id=range(1, 1 + len(morse)), observation_type_concept_id=32817)

obs.head(15)

,person_id,visit_occurrence_id,visit_detail_id,observation_date,observation_datetime,observation_concept_id,observation_source_value,value_as_string,value_as_concept_id,value_source_value,observation_id,observation_type_concept_id
0,108386,108386_m_1,12524022,2014-10-16,2014-10-16 14:25:00,2000018739,morse_ambulatory_aids,0,3542363,0,1,32817
1,108386,108386_m_1,12524022,2014-10-16,2014-10-16 19:41:00,2000018743,morse_mental_status,0,3542363,0,2,32817
2,108386,108386_m_1,12524022,2014-10-16,2014-10-16 14:25:00,2000018742,morse_intravenous,20,3542367,20,3,32817
3,108386,108386_m_1,12524022,2014-10-16,2014-10-16 19:41:00,2000018744,morse_score,20,3542367,20,4,32817
4,108386,108386_m_1,12524022,2014-10-15,2014-10-15 19:02:00,2000018745,morse_secondary_dx,0,3542363,0,5,32817
5,108386,108386_m_1,12524022,2014-10-19,2014-10-19 12:30:00,2000018741,morse_gait_transferring,0,3542363,0,6,32817
6,108386,108386_m_1,12524022,2014-10-18,2014-10-18 11:11:00,2000018740,morse_fall_hx,0,3542363,0,7,32817
7,108386,108386_m_1,12524022,2014-10-17,2014-10-17 09:15:00,3521695,fall_risk_indicator,Yes,3542372,Yes,8,32817
8,108395,108395_m_2,12524030,2015-02-04,2015-02-04 21:32:00,2000018743,morse_mental_status,0,3542363,0,9,32817
9,108395,108395_m_2,12524030,2015-02-04,2015-02-04 21:32:00,2000018744,morse_score,0,3542363,0,10,32817


## 5.3 Target Table : Measurement

<h3 style='color:green'> About :</h3>
</br>
<div style="text-align: justify">

In the MEASUREMENT table, you will find data (structured numeric or categoric values) derived from a standardized examination related to a person, including test orders, test results, vital signs, and other measurements. Similar to Observations, Measurements can be recorded as an attribute paired with a value, with the attribute defining the measurement concept and the value defining the measurement result. The result can be either a concept (VALUE_AS_CONCEPT) or a number (VALUE_AS_NUMBER) and a unit (UNIT_CONCEPT_ID). In the PROCEDURE_OCCURRENCE table, you may find the Procedure for acquiring the sample, but it is not a requirement for measurements that do not have a PROCEDURE_OCCURRENCE in their source data. In contrast to Observations, Measurements must stem from a standardized exam that creates a measurable result. If that result is not recorded, the assumption is that the exam took place but no result was captured. Most of the time, Measurements are lab tests, but Measurements can also be results of function tests or blood pressure tests. As you investigate Measurements, be sure to search for operator_concept_ids (such as < and >).


</div>
<h3 style='color:green'>ETL Coversions :</h3>
</br>
<div style="text-align: justify">
The MEASUREMENT table should only include records that have a source value mapped to a Measurement Concept. Though by definition every Measurement must stem from a standardized exam that yields a measurable result, the fields VALUE_AS_NUMBER and VALUE_AS_CONCEPT_ID are not required because results are often not provided in the source data. In the case of an unknown result, the Measurement record simply indicates that the standardized exam was conducted. When a result is included in the source data—for instance, ICD10 CONCEPT_ID 45548980 “Abnormal level of unspecified serum enzyme”—contains a Measurement value and the result of “abnormal.” In this example, the CONCEPT_RELATIONSHIP table includes both a “Maps to” record and another record with the relationship_id of “Maps to value.” The “Maps to” record would direct to 4046263 “Enzyme measurement” and the “Maps to value” record would direct to 4135493 “Abnormal.”
</div>

Refer to [click here](https://ohdsi.github.io/CommonDataModel/cdm54.html#MEASUREMENT) for more details about Measurement table

<h3 style='color:green'>Source Table : <h3 style='color:red;align:inline'>1) Heart Rate</h3></h3>

<h3 style='color:green'>About :</h3>
</br>
<div style="text-align: justify">
Heart Rate table is the source table which holds fields like heart-rate and source used to calculate along with dates which needs to be mapped to CDM Measurement table as target. 
</div>

<h3 style='color:green'>Mapping :</h3>

![heart_rate_2_measurements](./Images/hr_2_meas.png)

In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

#Parse dates of this format from input file to log entries in target table
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

#Heart rate source table
hr = pd.read_csv('./SampleDatasets/heart_rate_sample.csv',parse_dates=['vitals_datetime'], date_parser=dateparse)
hr.head(10)

,patient_deiden_id,merged_enc_id,encounter_deiden_id,vitals_datetime,heart_rate,heart_rate_source
0,108386,108386_m_1,12524022,2014-10-18 08:50:00,63,NaN
1,108386,108386_m_1,12524022,2014-10-18 10:01:00,70,NaN
2,108386,108386_m_1,12524022,2014-10-18 10:02:00,69,NaN
3,108386,108386_m_1,12524022,2014-10-18 10:03:00,64,NaN
4,108386,108386_m_1,12524022,2014-10-18 10:05:00,57,NaN
5,108386,108386_m_1,12524022,2014-10-18 10:06:00,58,NaN
6,108386,108386_m_1,12524022,2014-10-18 10:06:00,56,NaN
7,108386,108386_m_1,12524022,2014-10-18 10:07:00,60,NaN
8,108386,108386_m_1,12524022,2014-10-18 10:07:00,61,NaN
9,108386,108386_m_1,12524022,2014-10-18 10:01:00,71,NaN


In [6]:
#Target table
meas=pd.DataFrame().assign(person_id=hr['patient_deiden_id'], visit_occurrence_id=hr['merged_enc_id'], visit_detail_id=hr['encounter_deiden_id'],
    measurement_date = hr['vitals_datetime'].dt.strftime("%Y-%m-%d"),
    measurement_time = hr['vitals_datetime'].dt.strftime("%H:%M:%S"),
    measurement_datetime = hr['vitals_datetime'],
    measurement_concept_id = '8867-4', ## LOINC code for heart rate
    measurement_source_value = 'Heart Rate',
    value_as_number=  hr['heart_rate'],              
    value_source_value =hr['heart_rate_source']+" Heart Rate"
                          )
meas.head(10)

,person_id,visit_occurrence_id,visit_detail_id,measurement_date,measurement_time,measurement_datetime,measurement_concept_id,measurement_source_value,value_as_number,value_source_value
0,108386,108386_m_1,12524022,2014-10-18,08:50:00,2014-10-18 08:50:00,8867-4,Heart Rate,63,NaN
1,108386,108386_m_1,12524022,2014-10-18,10:01:00,2014-10-18 10:01:00,8867-4,Heart Rate,70,NaN
2,108386,108386_m_1,12524022,2014-10-18,10:02:00,2014-10-18 10:02:00,8867-4,Heart Rate,69,NaN
3,108386,108386_m_1,12524022,2014-10-18,10:03:00,2014-10-18 10:03:00,8867-4,Heart Rate,64,NaN
4,108386,108386_m_1,12524022,2014-10-18,10:05:00,2014-10-18 10:05:00,8867-4,Heart Rate,57,NaN
5,108386,108386_m_1,12524022,2014-10-18,10:06:00,2014-10-18 10:06:00,8867-4,Heart Rate,58,NaN
6,108386,108386_m_1,12524022,2014-10-18,10:06:00,2014-10-18 10:06:00,8867-4,Heart Rate,56,NaN
7,108386,108386_m_1,12524022,2014-10-18,10:07:00,2014-10-18 10:07:00,8867-4,Heart Rate,60,NaN
8,108386,108386_m_1,12524022,2014-10-18,10:07:00,2014-10-18 10:07:00,8867-4,Heart Rate,61,NaN
9,108386,108386_m_1,12524022,2014-10-18,10:01:00,2014-10-18 10:01:00,8867-4,Heart Rate,71,NaN


<h3 style='color:green'>Source Table : <h3 style='color:red;align:inline'> 2) Temperature</h3></h3>

<h3 style='color:green'>About :</h3>
</br>
<div style="text-align: justify">
Temperature table is the source table which holds details about the temperatures recorded, units used and dates which needs to be mapped to CDM Measurement table as target. 
</div>

<h3 style='color:green'>Mapping :</h3> </br>Temperature to Measurement

![temperature_2_measurements](./Images/temp_2_meas.png)


In [7]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

#Parse dates of this format from input file to log entries in target table
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M')

#Temperature source table
temp = pd.read_csv('./SampleDatasets/temperature_sample.csv',parse_dates=['vitals_datetime'], date_parser=dateparse)
temp.head(10)

,patient_deiden_id,merged_enc_id,encounter_deiden_id,vitals_datetime,temp_source_value,temp_source_unit,temp_celsius,clean_core_body_temp_celsius
0,108386,108386_m_1,12524022,2014-10-15 18:46:00,98.1,Farenheit,36.7222,37.2222
1,108386,108386_m_1,12524022,2014-10-18 13:23:00,97.4,Farenheit,36.3333,36.8333
2,108386,108386_m_1,12524022,2014-10-18 13:22:00,97.4,Farenheit,36.3333,36.8333
3,108386,108386_m_1,12524022,2014-10-18 13:21:00,97.4,Farenheit,36.3333,36.8333
4,108386,108386_m_1,12524022,2014-10-18 13:20:00,97.4,Farenheit,36.3333,36.8333
5,108386,108386_m_1,12524022,2014-10-18 13:19:00,97.4,Farenheit,36.3333,36.8333
6,108386,108386_m_1,12524022,2014-10-18 13:18:00,97.4,Farenheit,36.3333,36.8333
7,108386,108386_m_1,12524022,2014-10-18 13:17:00,97.4,Farenheit,36.3333,36.8333
8,108386,108386_m_1,12524022,2014-10-18 13:16:00,97.4,Farenheit,36.3333,36.8333
9,108386,108386_m_1,12524022,2014-10-18 13:15:00,97.5,Farenheit,36.3889,36.8889


In [8]:
#Target table
meas=pd.DataFrame().assign(person_id=temp['patient_deiden_id'], visit_occurrence_id=temp['merged_enc_id'], visit_detail_id=temp['encounter_deiden_id'],
    measurement_date = temp['vitals_datetime'].dt.strftime("%Y-%m-%d"),
    measurement_time = temp['vitals_datetime'].dt.strftime("%H:%M:%S"),
    measurement_datetime = temp['vitals_datetime'],
    measurement_concept_id = '8310-5', # LOINC code for body temperature
    measurement_source_value = 'body temperature',
    value_as_number=  round(temp['clean_core_body_temp_celsius'], 2),              
    value_source_value =temp["clean_core_body_temp_celsius"].astype(str).replace('nan','')+ " celsius")
              
meas.head(10)

,person_id,visit_occurrence_id,visit_detail_id,measurement_date,measurement_time,measurement_datetime,measurement_concept_id,measurement_source_value,value_as_number,value_source_value
0,108386,108386_m_1,12524022,2014-10-15,18:46:00,2014-10-15 18:46:00,8310-5,body temperature,37.22,37.2222 celsius
1,108386,108386_m_1,12524022,2014-10-18,13:23:00,2014-10-18 13:23:00,8310-5,body temperature,36.83,36.8333 celsius
2,108386,108386_m_1,12524022,2014-10-18,13:22:00,2014-10-18 13:22:00,8310-5,body temperature,36.83,36.8333 celsius
3,108386,108386_m_1,12524022,2014-10-18,13:21:00,2014-10-18 13:21:00,8310-5,body temperature,36.83,36.8333 celsius
4,108386,108386_m_1,12524022,2014-10-18,13:20:00,2014-10-18 13:20:00,8310-5,body temperature,36.83,36.8333 celsius
5,108386,108386_m_1,12524022,2014-10-18,13:19:00,2014-10-18 13:19:00,8310-5,body temperature,36.83,36.8333 celsius
6,108386,108386_m_1,12524022,2014-10-18,13:18:00,2014-10-18 13:18:00,8310-5,body temperature,36.83,36.8333 celsius
7,108386,108386_m_1,12524022,2014-10-18,13:17:00,2014-10-18 13:17:00,8310-5,body temperature,36.83,36.8333 celsius
8,108386,108386_m_1,12524022,2014-10-18,13:16:00,2014-10-18 13:16:00,8310-5,body temperature,36.83,36.8333 celsius
9,108386,108386_m_1,12524022,2014-10-18,13:15:00,2014-10-18 13:15:00,8310-5,body temperature,36.89,36.8889 celsius


## 5.4 Target Table : Visit Occurence

<h3 style='color:green'> About :</h3>
</br>
<div style="text-align: justify">

In the VISIT OCCURRENCE table, you will find Events (often called “encounters”) that describe a Person’s engagement with the healthcare system. These Visits are defined by circumstances such as (1) where the visit takes place (in a healthcare facility, at home, remotely), (2) what medical staff is administering the medical treatment or consultation during the visit, and (3) the duration of the visit (a brief exchange or a longer-term period that involves the patient staying in bed). The values VISIT_END_DATE and VISIT_START_DATE are used to determined the length of the visit. Besides Inpatient Visits and Non-Hospital Institution Visits, all other Visits are less than a 24-hour-period. Additional details about the Visit is stored in the CDM’s VISIT_DETAIL table, which could include information about unit transfers during inpatient Visits, etc.

</div>
<h3 style='color:green'>ETL Coversions :</h3>
</br>
<div style="text-align: justify">
    
If the source data has a code for Place of Service or for Procedures (such as a CPT code for a well visit), then Visits can be derived by looking up the code and mapping it to a Standard Visit Concept. If the source data does not contain such coding systems, then the ETL process must identify the Visit Concepts. In this table, the Visit Concepts are ordered hierarchically in order to allow cohort definitions by rolling up to Visits that are typically in use by a majority of healthcare systems. Visits may be adjacent (meaning that one end date can be the same as the next start date); therefore, more than one Visit may be record for a single day. Visits spanning more than one day cannot overlap; one multi-day visit can only share its end date with another multi-day visit’s start date. Frequently, logic is written to define Visits and to define how a Visit_Concept_ID should be assigned. In claims processing in the United States, for example, outpatient visits that happen during the same period as an inpatient visit can be combined into a single Visit_Occurrence_ID. With EHR data, inpatient visits that happen within a day of one another can be combined into a single Visit. The logic on how visits should be written will depend on the source data and the way in which the encounters need to be translated into visits. To associate a provider with a Visit, the PROVIDER_ID field can be used or the PROCEDURE_OCCURRENCE records can link to both the VISIT table and the PROVIDER table.
</div>

Refer to [click here](https://ohdsi.github.io/CommonDataModel/cdm54.html#VISIT_OCCURRENCE) for more details about Visit Occurrence table

<h3 style='color:green'>Source Table : <h3 style='color:red;align:inline'> Encounters</h3></h3>

<h3 style='color:green'>About :</h3>
</br>
<div style="text-align: justify">
Encounters table is the source table which holds patient's details like number of visits, date of visit and purpose which needs to be mapped to CDM Visit Occurrence table as target.
</div>

<h3 style='color:green'>Mapping :</h3>

![encounters_2_visit_occurrence](./Images/enc_2_visit_occ.png)

In [9]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

#Parse dates of this format from input file to log entries in target table
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

#Source encounters table
en = pd.read_csv('./SampleDatasets/encounters_sample.csv', usecols = ['patient_deiden_id', 'merged_enc_id', 'merged_admit_datetime', 'encounter_type', 'patient_type', 'external_name','hospital', 'location_of_svc', 'encounter_effective_date', 'merged_dischg_datetime', 'attending_dr_deiden_id', 'admit_source', 'dischg_disposition'], \
                 parse_dates=['merged_admit_datetime','merged_dischg_datetime','encounter_effective_date'], date_parser=dateparse)
en.head(10)

,patient_deiden_id,merged_enc_id,merged_admit_datetime,encounter_type,patient_type,encounter_effective_date,merged_dischg_datetime,attending_dr_deiden_id,location_of_svc,hospital,admit_source,external_name,dischg_disposition
0,114927,114927_m_1,2017-02-02 15:58:00,HOSPITAL ENCOUNTER,OBSERVATION,2017-02-02 03:20:00,2017-02-03 14:46:00,32391,UF UA6E,UF,NON HEALTHCARE FACILITY,UF UA6E,TO HOME
1,114920,114920_m_1,2017-03-30 09:16:00,HOSPITAL ENCOUNTER,AMBULATORY SURGERY,2017-03-30 03:20:00,2017-03-30 13:31:00,34191,UF UAOR,UF,NON HEALTHCARE FACILITY,UF UAOR,TO HOME
2,114985,114985_m_1,2017-04-05 08:44:00,HOSPITAL ENCOUNTER,INPATIENT,2017-04-05 03:20:00,2017-04-08 14:59:00,15899,UF UA6E,UF,NON HEALTHCARE FACILITY,UF UA6E,TO HOME
3,114930,114930_m_3,2016-02-15 09:39:00,HOSPITAL ENCOUNTER,AMBULATORY SURGERY,2016-02-15 03:20:00,2016-02-15 12:46:00,35381,UF GI ENDOSCOPY,UF,CLINIC OR PHYSICIAN REFERRAL,UF GI ENDOSCOPY,TO HOME
4,114931,114931_m_1,2017-01-20 12:45:00,HOSPITAL ENCOUNTER,AMBULATORY SURGERY,2017-01-20 03:20:00,2017-01-20 21:05:00,21289,UF UAOR,UF,NON HEALTHCARE FACILITY,UF UAOR,TO HOME
5,114917,114917_m_1,2019-01-13 04:53:00,HOSPITAL ENCOUNTER,INPATIENT,2019-01-13 03:20:00,2019-01-17 19:04:00,20083,UF UA6E,UF,NON HEALTHCARE FACILITY,UF UA6E,TO HOME
6,114977,114977_m_1,2014-11-01 04:44:00,HOSPITAL ENCOUNTER,INPATIENT,2014-11-01 03:20:00,2014-12-30 19:20:00,17383,UF UA6E,UF,NON HEALTHCARE FACILITY,UF UA6E,TO REHAB
7,114918,114918_m_1,2016-06-17 12:01:00,HOSPITAL ENCOUNTER,INPATIENT,2016-06-17 03:20:00,2016-06-23 16:17:00,10768,UF 64MS,UF,NON HEALTHCARE FACILITY,UF 64MS,TO HOME
8,114918,114918_m_2,2016-08-12 10:02:00,HOSPITAL ENCOUNTER,AMBULATORY SURGERY,2016-08-12 03:20:00,2016-08-12 18:01:00,21289,UF NTOR,UF,NON HEALTHCARE FACILITY,UF NTOR,TO HOME
9,114978,114978_m_1,2016-12-20 12:11:00,HOSPITAL ENCOUNTER,AMBULATORY SURGERY,2016-12-20 03:20:00,2016-12-20 19:34:00,14900,UF GI ENDOSCOPY,UF,CLINIC OR PHYSICIAN REFERRAL,UF GI ENDOSCOPY,TO HOME


In [10]:
#Dictionaries to map concept_ids to respective field values in target table from source
enc_type={"HOSPITAL ENCOUNTER": 25}
pat_type={"INPATIENT": 2432,"AMBULATORY SURGERY":2433,"OUTPATIENT":2434,"EMERGENCY":2435,"OBSERVATION":2436,"RECURRING OUTPATIENT":2437}
dis_dis={"TO HOMECARE":10002,"TO HOME":10003,"EXPIRED AUT UNK":10004,"TO SKILLED NURSING":10005,"EXPIRED AUT UNK":10006,"TO COURT OR LAW ENFORCEMENT":10007,"TO REHAB":10008,
         "TO ANOTHER HOSPITAL":10009,"AMA":10010,"TO HOSPICE FACILITY":10011,"TO HOSPICE HOME":10012,"TO REHAB PLANNED READMIT":10013,"TO LONG TERM CARE HOSPITAL":10014}
adm_src={'NON HEALTHCARE FACILITY':3241, 'FROM ANOTHER HOSPITAL':3242,'CLINIC OR PHYSICIAN REFERRAL':3243, 'FROM ANOTHER TYPE OF HEALTHCARE FACILITY':3244,
 'ROUTINE SCHEDULED':3245, 'OUTPATIENT SERVICE':3246,'INTERNAL TRANSFER':3247,'FROM SKILLED NURSING OR ICF':3248, 'COURT OR LAW ENFORCEMENT':3249, 'FROM SHANDS UF':3250,'ROUTINE UNSCHEDULED':3251}

#Target table mapping with ETL conversions
vst_occ=pd.DataFrame().assign(person_id=en['patient_deiden_id'], visit_occurrence_id=en['merged_enc_id'], visit_occurrence_source_id= en['merged_enc_id'], provider_id=en['attending_dr_deiden_id'],
visit_end_datetime= np.where(en['merged_dischg_datetime'], en['merged_dischg_datetime'],en['encounter_effective_date']),
visit_end_date=np.where(en['merged_dischg_datetime'], en['merged_dischg_datetime'].dt.strftime("%Y-%m-%d"),en['encounter_effective_date'].dt.strftime("%Y-%m-%d")),
visit_type_concept_id= 32817, visit_start_datetime= np.where(en['merged_admit_datetime'], en['merged_admit_datetime'],en['encounter_effective_date']),
visit_start_date=np.where(en['merged_admit_datetime'], en['merged_admit_datetime'].dt.strftime("%Y-%m-%d"),en['encounter_effective_date'].dt.strftime("%Y-%m-%d")),
visit_concept_id=en['encounter_type'].str.extract(fr"({'|'.join(enc_type.keys())})", expand=False).map(enc_type)+en['patient_type'].str.extract(fr"({'|'.join(pat_type.keys())})", expand=False).map(pat_type),
visit_source_value=en['encounter_type']+":"+en['patient_type']+":"+en["external_name"]+":"+en["hospital"],
care_site_id=en['hospital']+en['location_of_svc'], discharged_to_concept_id=en['dischg_disposition'].str.extract(fr"({'|'.join(dis_dis.keys())})", expand=False).map(dis_dis),
discharged_to_source_value=en['dischg_disposition'],admitted_from_source_value=en['admit_source'], admitted_from_concept_id=en['admit_source'].str.extract(fr"({'|'.join(adm_src.keys())})", expand=False).map(adm_src))
vst_occ.head(10)

,person_id,visit_occurrence_id,visit_occurrence_source_id,provider_id,visit_end_datetime,visit_end_date,visit_type_concept_id,visit_start_datetime,visit_start_date,visit_concept_id,visit_source_value,care_site_id,discharged_to_concept_id,discharged_to_source_value,admitted_from_source_value,admitted_from_concept_id
0,114927,114927_m_1,114927_m_1,32391,2017-02-03 14:46:00,2017-02-03,32817,2017-02-02 15:58:00,2017-02-02,2461,HOSPITAL ENCOUNTER:OBSERVATION:UF UA6E:UF,UFUF UA6E,10003,TO HOME,NON HEALTHCARE FACILITY,3241
1,114920,114920_m_1,114920_m_1,34191,2017-03-30 13:31:00,2017-03-30,32817,2017-03-30 09:16:00,2017-03-30,2458,HOSPITAL ENCOUNTER:AMBULATORY SURGERY:UF UAOR:UF,UFUF UAOR,10003,TO HOME,NON HEALTHCARE FACILITY,3241
2,114985,114985_m_1,114985_m_1,15899,2017-04-08 14:59:00,2017-04-08,32817,2017-04-05 08:44:00,2017-04-05,2457,HOSPITAL ENCOUNTER:INPATIENT:UF UA6E:UF,UFUF UA6E,10003,TO HOME,NON HEALTHCARE FACILITY,3241
3,114930,114930_m_3,114930_m_3,35381,2016-02-15 12:46:00,2016-02-15,32817,2016-02-15 09:39:00,2016-02-15,2458,HOSPITAL ENCOUNTER:AMBULATORY SURGERY:UF GI EN...,UFUF GI ENDOSCOPY,10003,TO HOME,CLINIC OR PHYSICIAN REFERRAL,3243
4,114931,114931_m_1,114931_m_1,21289,2017-01-20 21:05:00,2017-01-20,32817,2017-01-20 12:45:00,2017-01-20,2458,HOSPITAL ENCOUNTER:AMBULATORY SURGERY:UF UAOR:UF,UFUF UAOR,10003,TO HOME,NON HEALTHCARE FACILITY,3241
5,114917,114917_m_1,114917_m_1,20083,2019-01-17 19:04:00,2019-01-17,32817,2019-01-13 04:53:00,2019-01-13,2457,HOSPITAL ENCOUNTER:INPATIENT:UF UA6E:UF,UFUF UA6E,10003,TO HOME,NON HEALTHCARE FACILITY,3241
6,114977,114977_m_1,114977_m_1,17383,2014-12-30 19:20:00,2014-12-30,32817,2014-11-01 04:44:00,2014-11-01,2457,HOSPITAL ENCOUNTER:INPATIENT:UF UA6E:UF,UFUF UA6E,10008,TO REHAB,NON HEALTHCARE FACILITY,3241
7,114918,114918_m_1,114918_m_1,10768,2016-06-23 16:17:00,2016-06-23,32817,2016-06-17 12:01:00,2016-06-17,2457,HOSPITAL ENCOUNTER:INPATIENT:UF 64MS:UF,UFUF 64MS,10003,TO HOME,NON HEALTHCARE FACILITY,3241
8,114918,114918_m_2,114918_m_2,21289,2016-08-12 18:01:00,2016-08-12,32817,2016-08-12 10:02:00,2016-08-12,2458,HOSPITAL ENCOUNTER:AMBULATORY SURGERY:UF NTOR:UF,UFUF NTOR,10003,TO HOME,NON HEALTHCARE FACILITY,3241
9,114978,114978_m_1,114978_m_1,14900,2016-12-20 19:34:00,2016-12-20,32817,2016-12-20 12:11:00,2016-12-20,2458,HOSPITAL ENCOUNTER:AMBULATORY SURGERY:UF GI EN...,UFUF GI ENDOSCOPY,10003,TO HOME,CLINIC OR PHYSICIAN REFERRAL,3243


> The documentation is generated based on [OHDSI official websites](https://www.ohdsi.org/) and [OHDSI cookbook](https://ohdsi.github.io/TheBookOfOhdsi/).